# Prepare test data for annotation

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#standard libs
import os, sys
from pathlib import Path
from pprint import pprint
import random
import json
import itertools
from datetime import datetime as dt
# ds libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm, tqdm_notebook
# custom path
os.chdir('..')

In [72]:
from collections import Counter
import re

In [2]:
from src.train.predict import prepare_texts, load_models, load_test_file, format_text, predict_language


In [35]:
get_file_lang = lambda fp: Path(fp).stem.split('-')[2]

## New randomly selected tasks

In [36]:
FILES = [
    'data/external/dc0415-input/original/dc0415-input-ru.txt',
    'data/external/dc0415-input/original/dc0415-input-en.txt',
    'data/external/dc0415-input/translated/dc0415-input-ar-translated.txt',
    'data/external/dc0415-input/translated/dc0415-input-fa-translated.txt',
    'data/external/dc0415-input/translated/dc0415-input-uz-translated.txt',
]

### Load

In [37]:
test_data = {
    get_file_lang(f): random.sample(load_test_file(f, verbose=False), 100)
    for f in FILES
            }

for l,d in test_data.items():
    print(l, len(d))

ru 100
en 100
ar 100
fa 100
uz 100


### explore

In [42]:
l = random.choice(list(test_data.keys()))
d = random.choice(test_data[l])

print(f"Language {l.upper()}")
pprint(d)

Language UZ
{'counters': {'audios': 129,
              'files': 92,
              'photos': 268,
              'posts': 3228,
              'videos': 71},
 'description': 'Assalomu alaykum ❗ Here you will have Eng️ 1️⃣ Latest Tests '
                'Qu✅ 2️⃣ Best Quizzes💡✅ 3️⃣ Most Necessary Guides 📑✅ 4️⃣ '
                'Latest Information atsiya✅ 5️⃣ Best Motivations 📈🗝✅ Contact : '
                '@Faylasufginam Tel number: +998993433211',
 'recent_posts': [{'text': '🎲 “GRADE 11 UZBEK HISTORY CHANNEL: '
                           '@HISTORY_QUIZ_UZ Facts: @HISTORY_MEMORY Channel: '
                           '@GRANT_UZ Compiler: @ Jurist_19” test 🖊 15 '
                           'questions · ⏱ 30 seconds',
                   'type': 'text'},
                  {'text': '🎲 “📊GRANT FOR HISTORIANS Channel @ History_Quiz_Uz '
                           'Compiler: @Dilqushim” test test 10 questions · ⏱ '
                           '30 seconds',
                   'type': 'text'},
     

### Convert to tasks

In [135]:
def update_dict(src_dict,dest_dict):
    src_dict.update(dest_dict)
    return src_dict


def get_task_stats(tasks):
    print('# tasks', len(tasks))
    by_lang = Counter(t['lang_code'] for t in tasks)
    pprint(by_lang)

In [136]:
tasks = [update_dict(one, {'lang_code':lang}) for lang,channels in test_data.items() for one in channels]



In [137]:
get_task_stats(tasks)

# tasks 500
Counter({'ru': 100, 'en': 100, 'ar': 100, 'fa': 100, 'uz': 100})


In [138]:
random.choice(tasks)

{'title': 'Tehran Times | Tehran timse',
 'description': '🔻The most up-to-date news of Iran and the world باط Contact with reporters (regarding posts and content) @Ma_matrix',
 'subscribers': 78820,
 'counters': {'posts': 43545,
  'photos': 22951,
  'videos': 7894,
  'audios': 14,
  'files': 41},
 'recent_posts': [{'type': 'photo',
   'text': '🌀 The bell of Shams Al-Amara clock rang in Golestan Palace 🔹 The old clock of Tehran Golestan Palace, which is mounted on the Shams Al-Amara tower, sounded again after many years. 🔹 The sound of the bell of this old clock, which is the gift of Queen Victoria, Queen of England to Nasser al-Din Shah Qajar, wrapped up in the 12th district of Tehran this morning, Monday, April 13, 1400, and caused happiness. 🔹 The reason for the failure of the bell was its loud ringing and disturbing the residents of Golestan Palace. At the order of Nasser al-Din Shah, they try to turn down the alarm, but completely destroy it and turn off the clock and alarm togethe

### save tasks

In [139]:
today_str = dt.strftime(dt.today(), '%Y%m%d')
print(today_str)

20210420


In [140]:
random.shuffle(tasks)

with open(f'data/processed/labelling_tasks_{today_str}.json', 'w') as f:
    json.dump(tasks, f)

## TG labelled data

for reference when labelling

In [141]:
LABELLED_FILES = [
    {
        'input': 'data/external/submission1489/category_en/input.txt',
        'output': 'data/external/submission1489/category_en/tg_output.txt'
    },
    {
        'input': 'data/external/submission1489/category_ru/input.txt',
        'output': 'data/external/submission1489/category_ru/tg_output.txt'
    },
]

In [142]:
get_lang = lambda fp: Path(fp).parent.stem.split('_')[1]

### load files

In [143]:
labelled = {
    get_lang(files['input']): {k: load_test_file(file) for k,file in files.items()}
    for files in LABELLED_FILES
}

Loaded 14775 rows
Loaded 14775 rows
Loaded 16726 rows
Loaded 16726 rows


In [144]:
l = random.choice(list(labelled.keys()))

i = random.randint(0, len(labelled[l]['input']))
inp = labelled[l]['input'][i]
out = labelled[l]['output'][i]

print('Lang', l.upper(),)
print('\nINPUT:')
pprint(inp)
print('\nOUTPUT:')
pprint(out)

Lang RU

INPUT:
{'description': '',
 'recent_posts': ['Игорь еб*ло разобьёт САНЕ БИ '
                  'https://www.youtube.com/watch?v=lgYoR2YTIUU',
                  'Игорь и Марина расстанутся ? '
                  'https://www.youtube.com/watch?v=efeurlPH1yA',
                  'Мелстрой БОИТСЯ АХАХХА '
                  'https://www.youtube.com/watch?v=omVn4AMT6MU',
                  'MELLSTROY удалил ИГОРЮ КАНАЛ? '
                  'https://www.youtube.com/watch?v=f8bPe-8KPXM',
                  'ИГОРЬ ПОСАДИТ МЕЛСТРОЯ '
                  'https://www.youtube.com/watch?v=eeW_333eUC8',
                  '100 р просто так ? Залетай по ссылке на 1 win '
                  '❤️❤️❤️https://1wiab.top/#j4bv   Регайся , вводи промо BASS '
                  'И СОТКА У ТЕБЯ В КАРМАНЕ ( выводи сразу) \n'
                  'А кто любит крутить ФРИ СПИНЫ В КАЗИКЕ БЕЗ ВАГЕРА ТО ВАМ '
                  'СЮДА  40 ФРИ СПИНОВ  ! ИЗИ ПОДНЯТЬ БАБКИ '
                  'https://bit.ly/2HQaUrK   ( ВВО

### Format to dialogs

In [149]:
def format_input(inp):
    inp['recent_posts'] = [{'text':p, 'author':'xxx'} for p in inp['recent_posts']]
    return inp

In [150]:
reference_tasks = [
    update_dict(format_input(inp), out)
   for lang,data in labelled.items()
    for inp,out in zip(data['input'], data['output'])
]

In [151]:
random.choice(reference_tasks)

{'title': 'Не фейсбук',
 'description': 'для радостей и нытья, инстаграм https://www.instagram.com/farkashalina/?hl=ru',
 'recent_posts': [{'text': 'в какой-то момент у меня сбился прицел полностью. я так долго думала о ремонте, что картинка начала распадаться на пиксели, стало вообще не понятно, что я хочу, как должна выглядеть квартира... Смотрела заново тысячи картинок – и не собиралось у меня. Пока я не нашла идеальный образ. Я не шучу! Стилистически будет примерно так, но посветлее )',
   'author': 'xxx'},
  {'text': 'Ну, а вообще приходите к в клабхаус ко мне, поболтаем. Только надо выбрать – секс или ремонт? )) Я там timbuktoo',
   'author': 'xxx'},
  {'text': 'Муж купил пионы. Мне и ЛЕЕ. На 14 февраля, ага. Чтобы ей было не обидно. Что вы до этого знали о детоцентризме?! Но какое коварство! Не шоколадный муссовый торт, а пионы!!! Очень красивые. И большая редкость в Израиле )',
   'author': 'xxx'},
  {'text': 'Если что, то это икея. Шпон дуба.', 'author': 'xxx'},
  {'text': 'А 

In [152]:
get_task_stats(reference_tasks)

# tasks 31501
Counter({'ru': 16726, 'en': 14775})


### Save 

In [153]:
random.shuffle(reference_tasks)

with open(f'data/processed/labelled_tgdata_{today_str}.json', 'w') as f:
    json.dump(reference_tasks, f)

## Save data and configs

In [94]:
from lxml import etree

### generate and save configs

In [95]:
def clean_topic(orig):
    orig = re.split(' [–-] ', orig)[0].strip()
    return orig

In [96]:
TOPICS = """
    Art & Design
    Bets & Gambling – includes sports bets
    Books
    Business & Entrepreneurship
    Cars & Other Vehicles
    Celebrities & Lifestyle
    Cryptocurrencies
    Culture & Events
    Curious Facts
    Directories of Channels & Bots
    Drug Sale
    Economy & Finance
    Education
    Erotic Content
    Fashion & Beauty
    Fitness
    Forgery – includes fake documents, fake money, etc.
    Food & Cooking
    Foreign Language Learning
    Hacked Accounts & Software – includes carding, passwords for subscription services, etc.
    Health & Medicine
    History
    Hobbies & Activities
    Home & Architecture
    Humor & Memes
    Investments
    Job Listings
    Kids & Parenting
    Marketing & PR
    Motivation & Self-development - includes inspirational quotes and poetry
    Movies
    Music
    Offers & Promotions – includes products or services for sale, unless they fall under the newly added categories
    Personal Data – includes doxxing, databases
    Pets
    Pirated Content – films, music, books, but not software
    Politics & Incidents
    Prostitution
    Psychology & Relationships
    Real Estate
    Recreation & Entertainment
    Religion & Spirituality
    Science
    Spam & Fake Followers – includes spam tools and services, boosting followers, likes, etc.
    Sports – includes e-sports
    Technology & Internet
    Travel & Tourism
    Video Games
    Weapon Sale
    Other
""".strip().split('\n')

TOPICS = [clean_topic(t) for t in TOPICS]

len(TOPICS)

50

In [97]:
xml = etree.Element('View')
# headers
_ = etree.SubElement(xml, "Header", size='4', value=f'$lang_code')
_ = etree.SubElement(xml, "Header", size='4', value=f'$title')
_ = etree.SubElement(xml, "Header", size='4', value=f'$description')
_ = etree.SubElement(xml, 'Paragraphs', name="posts", value="$recent_posts", nameKey="type", textKey="text")
# primary
_ = etree.SubElement(xml, "Header", size="6",
                          value="Primary topics")
_ = etree.SubElement(xml, "Filter", name="filter_primary",
                      toName='primary', hotkey="shift+f",
                      minlength="1", )
primary = etree.SubElement(xml, "Choices", name="primary", toName="posts",
                          showInline="true", choice='multiple', required='true')
_ = [etree.SubElement(primary, 'Choice', value=s, )
     for s in TOPICS]
# secondary
_ = etree.SubElement(xml, "Header", size="6",
                          value="Secondary topics")
_ = etree.SubElement(xml, "Filter", name="filter_secondary",
                      toName='secondary',
                      minlength="1", )
secondary = etree.SubElement(xml, "Choices", name="secondary", toName="posts",
                          showInline="true", choice='multiple', required='false')
_ = [etree.SubElement(secondary, 'Choice', value=s, ) for s in TOPICS]
# pretty string
etree.indent(xml, space='  ')
config = etree.tostring(xml, pretty_print=True,).decode('utf-8')

In [98]:
with open(f'data/processed/labelling_config_{today_str}.xml', 'w') as f:
    f.write(config)

print(config)

<View>
  <Header size="4" value="$lang_code"/>
  <Header size="4" value="$title"/>
  <Header size="4" value="$description"/>
  <Paragraphs name="posts" value="$recent_posts" nameKey="type" textKey="text"/>
  <Header size="6" value="Primary topics"/>
  <Filter name="filter_primary" toName="primary" hotkey="shift+f" minlength="1"/>
  <Choices name="primary" toName="posts" showInline="true" choice="multiple" required="true">
    <Choice value="Art &amp; Design"/>
    <Choice value="Bets &amp; Gambling"/>
    <Choice value="Books"/>
    <Choice value="Business &amp; Entrepreneurship"/>
    <Choice value="Cars &amp; Other Vehicles"/>
    <Choice value="Celebrities &amp; Lifestyle"/>
    <Choice value="Cryptocurrencies"/>
    <Choice value="Culture &amp; Events"/>
    <Choice value="Curious Facts"/>
    <Choice value="Directories of Channels &amp; Bots"/>
    <Choice value="Drug Sale"/>
    <Choice value="Economy &amp; Finance"/>
    <Choice value="Education"/>
    <Choice value="Erotic Cont